# Creation d'un dataset implicit feedback à partir du fichier u.data  de ml-100k 

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("ml-100k/u.data" , sep='\t',header=None) 
df.columns = ['user_id','item_id','rating','timestamp']

In [2]:
df = df[['user_id','item_id']]

In [3]:
df

,user_id,item_id
0,196,242
1,186,302
2,22,377
3,244,51
4,166,346
...,...,...
99995,880,476
99996,716,204
99997,276,1090
99998,13,225


In [4]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

# Matrice de notes
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    # ASTUCE : row[2] on veut que l'item et user 0 soit noté 1 pour bien distinguer les 1 des 0. A la fin, après exclusion des 0, on retranche 1
    # à toutes les données pour rétablir la situation
    ratings[row[1]-1, row[2]-1] = row[2]


In [5]:
ratings

array([[1., 2., 3., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 2., 0., ..., 0., 0., 0.]])

In [6]:

# On découpe en train/tests, en prenant 10 % notes
# de chaque utilisateur pour les placer en test
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        nb = np.count_nonzero(ratings[user, :])
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=nb//10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

train, test = train_test_split(ratings)

In [7]:
train.shape, test.shape

((943, 1682), (943, 1682))

In [8]:
train_df = pd.DataFrame(train, dtype = int)
train_df[-1]=train_df.index + 1
order_columns = sorted(train_df.columns.to_list())
train_df = train_df[order_columns]
new_train = train_df.to_numpy()

In [9]:
train_list = list(new_train)
train_list2 = []
for elms in train_list:
    x = [elm-1 for elm in list(elms)  if elm>0]
    train_list2.append(x)

In [10]:
test_df = pd.DataFrame(test, dtype = int)
test_df[-1]=test_df.index + 1
order_columns = sorted(test_df.columns.to_list())
test_df = test_df[order_columns]
new_test = test_df.to_numpy()
test_list = list(new_test)
test_list2 = []
for elms in test_list:
    x = [elm-1 for elm in list(elms)  if elm>0]
    test_list2.append(x)

In [11]:
#train_list2[:2]
#test_list2[:2]

In [12]:
len(train_list2[0])

246

In [13]:
def export(donnees,nom):
    file=open(nom,'w')
    for items in donnees:
        new_item = str(items).replace(',','').replace('[','').replace(']','')
        file.writelines(new_item+'\n')
    file.close()

In [14]:
export(train_list2,'train.txt')
export(test_list2,'test.txt')

In [15]:
!head -n 5 train.txt

0 0 1 3 4 5 6 7 9 10 11 12 14 15 17 18 19 20 21 23 24 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 58 59 60 61 62 63 64 65 66 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 105 106 107 108 109 110 111 112 113 114 115 116 119 120 121 122 123 124 125 126 127 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 199 200 201 202 204 205 206 207 208 209 210 212 213 214 215 216 218 219 220 221 223 224 225 226 229 230 232 233 234 235 236 237 238 240 241 242 243 244 245 246 247 248 249 250 251 253 255 256 257 259 261 263 264 265 266 267 268 269 270 271
1 0 9 12 13 18 24 49 99 110 126 236 241 250 254 256 257 268 271 272 273 275 276 277 278 279 280 282 283 284 285 286 28

In [16]:
!head test.txt

0 2 8 13 16 22 29 57 67 104 117 118 128 146 198 203 211 217 222 227 228 231 239 252 254 258 260 262
1 274 281 295 296 307 313
2 287 324 331 336 341
3 299 358
4 49 61 134 209 218 363 368 370 374 379 388 395 403 416 417 439 441
5 11 14 69 78 86 99 132 165 169 190 222 237 271 273 339 460 473 506 515 528 533
6 7 30 63 67 72 117 130 140 170 177 180 185 189 213 265 366 383 401 430 435 470 471 482 490 500 512 513 550 577 579 582 589 590 611 616 626 633 637 657 658
7 176 181 194 257 565
8 5 478
9 31 39 123 175 178 179 196 339 356 419 496 530 601 610 616 628 651 654
